# ローカルLLM(ここではrinna)からLlamaIndexを使う例
## 概要
[LlamaIndex](https://github.com/jerryjliu/llama_index)はLLMを外部データに紐付けるインタフェースを提供しているツールです。ChatGPTを直接使うだけだとその学習範囲だけからしか応答できませんが、独自に用意したデータに基づいて応答させるには毎回その文書を入力する必要が出てきます。ここでの大きな問題は、(1)プライバシーや守秘義務のあるデータをOpenAIのAPIに入力することは難しいこと、(2)仮に入力して良いことになったとしても毎回入力するのはコスト的に無駄が大きいことです。これらの問題を解決するため、ローカルでLLMを動かし、そのLLMを対象にインデックス生成するコード例を作ってみました。

なお、ChatGPT使って良い状況ならば以下の記事を参考にすると良いでしょう。というか何も考えずに公式コード例でも動作するはず。
- Fusic 技術ブログ
: [LlamaIndex で ChatGPT に専門知識を組み込んでみた](https://zenn.dev/fusic/articles/try-llamaindex)
- npaka note: [GPT Index で専門知識を必要とする質問応答チャットボットを簡単作成](https://note.com/npaka/n/nd23bdf33d929)
- LlamaIndex
    - [Tutorial](https://gpt-index.readthedocs.io/en/latest/guides/tutorials.html)
    - [Example: Using a Custom LLM Model](https://gpt-index.readthedocs.io/en/latest/how_to/customization/custom_llms.html#example-using-a-custom-llm-model), facebook/opt-iml-max-30b使う例なので注意。

---
## やったこと、やれなかったこと
以下ではローカルLLMとして[rinna/japanese-gpt2-medium](https://huggingface.co/rinna/japanese-gpt2-medium)を使っています。外部データは ./data/ にテキストファイルを数件用意しました。これらのテキストファイルに対するindexを作成し、[query実行する](https://gpt-index.readthedocs.io/en/latest/use_cases/queries.html)ことで適切なファイルを参照して応答文生成するところまでは確認できました。ただしモデルが小さいために応答文そのものは適切な結果が得られていません。ただし、「履修時に気をつけることはありますか？」という質問に対して「1年間の取得単位数が16未満だと除籍になります。」をtop 1で参照できているのは結構良さげ。とはいっても今回は極端に違うテキストファイルを用意したからでもありますが。

なお本来やりたかったことは[GPT4All](https://github.com/nomic-ai/gpt4all)をローカルLLMとして使うことです。ローカルで動かし、モデルとして読み込むところまではいきましたが、embedding生成ができませんでした。embeddingなしでもquery実行できるようなのですが、これが何をどう処理しているのかはまだ理解できていません。

---
## Tips
- device指定をcpuにしています。これは手元のノートPCではcuda:0を実行できず、またmpsでも一部の問題があり動作しない部分があったためです。
- LlamaIndex触る場合には、既にChatGPTをAPI利用している人はそれを利用できないようにしておく（環境変数設定してるならunsetする）ことをオススメします。ローカルLLM指定する箇所が複数あり、一つでも漏れがあると「勝手にOpenAIのAPI呼びに行く」ことがあるためです。ローカルで動いているように見せかけてOpenAIに投げてたら泣けますよね。
- indexは複数フォーマットが用意されています。チュートリアルではGPTListIndex, GPTSipmleVectorIndexあたりが出てきます。が、何やら llama-index==0.5.10 時点では動作が怪しいです。具体的には GPTListIndex では OpenAI API 参照するのを変更することができませんでした。指定することはできるのですが、効かない。
- 作成したindexは [BaseGPTIndex.save_to_disk](https://gpt-index.readthedocs.io/en/latest/reference/indices.html#gpt_index.indices.base.BaseGPTIndex.save_to_disk) でJSON形式で保存することができます。ここで気づいたのですが、index作成するチュートリアルをそのまま実行するだけでは外部データのembeddingを作成してくれません。別途指定する必要があります。

---
## 動作確認した環境
- PC: macOS Ventura 13.3, Apple M1, 16GB
- Python 3.9.6
- pip + venvで以下のように環境構築
```shell
pip install --upgrade pip
pip install jupyter transformers llama-index torch sentence_transformers "protobuf==3.20.*"
```
- transformers==4.27.4
- llama-index==0.5.10
- torch==2.0.0
- sentence-transformers==2.2.2
- protobuf==3.20.3

In [1]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms.base import LLM
from llama_index import LangchainEmbedding, ServiceContext
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTSimpleVectorIndex, PromptHelper, LLMPredictor, ServiceContext
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from typing import Optional, List, Mapping, Any

import langchain
langchain.verbose = True # 詳細ログ確認するため

model_name = 'rinna/japanese-gpt2-medium'

class CustomLLM(LLM):
    text_generator = pipeline("text-generation", model=model_name, device="cpu", model_kwargs={"torch_dtype":torch.bfloat16})
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = self.text_generator(prompt, max_new_tokens=num_output)[0]["generated_text"]
        
        # only return newly generated tokens
        return response[prompt_length:]
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": self.model_name}
    
    @property
    def _llm_type(self) -> str:
        return "custom"

# set maximum input size: 外部テキスト + プロンプトの最大トークン数指定になっていそう。
max_input_size = 256
# set number of output tokens
num_output = 128
# set maximum chunk overlap
max_chunk_overlap = 20

tokenizer = AutoTokenizer.from_pretrained(model_name)
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap, tokenizer=tokenizer)
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name=model_name))
llm = AutoModelForCausalLM.from_pretrained(model_name)

service_context = ServiceContext.from_defaults(llm_predictor=LLMPredictor(llm=CustomLLM()), embed_model=embed_model, prompt_helper=prompt_helper)

# ドキュメントの読み込み
documents = SimpleDirectoryReader('data').load_data()
index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)

# index.save_to_dict('index.json') # for your demand

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: rinna/japanese-gpt2-medium
Some weights of the model checkpoint at /Users/tnal/.cache/torch/sentence_transformers/rinna_japanese-gpt2-medium were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 335 tokens


## クエリの実行例
"Context information is below" 以下の線で囲われたテキストは、外部データとして用意した複数テキストから一つ選んできたテキスト本文を出力しています。

In [2]:
query = "naltomaの正体は？"
output = index.query(query, similarity_top_k=1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/Users/tnal/.venv/flexgen/lib/python3.9/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
Context information is below. 
---------------------
naltomaの正体はマスクマンです。

---------------------
Given the context information and not prior knowledge, answer the question: naltomaの正体は？



INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 100 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 12 tokens



> Finished chain.


In [3]:
output

Response(response="iven. them not important of the prix with---------------------------- iven [--------------iven the context information and not including compactes. ] -------- t. i'm proudly not produced in others of miracle monsters (---------", source_nodes=[NodeWithScore(node=Node(text='naltomaの正体はマスクマンです。\n', doc_id='c4c26c6f-4ec8-4c40-aae0-a8759c523bbf', embedding=None, doc_hash='1bbdd5e7be382989a7785441fbf29538b11d7358df2242acf7fc4dfeaea35436', extra_info=None, node_info={'start': 0, 'end': 20}, relationships={<DocumentRelationship.SOURCE: '1'>: '13396714-b234-493a-a2e3-19d060a36d93'}), score=0.9216100484418455)], extra_info=None)

In [4]:
print(output)

iven. them not important of the prix with---------------------------- iven [--------------iven the context information and not including compactes. ] -------- t. i'm proudly not produced in others of miracle monsters (---------


In [5]:
query = "履修時に気をつけることはありますか？"
output = index.query(query, similarity_top_k=1)
print(output)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
Context information is below. 
---------------------
1年間の取得単位数が16未満だと除籍になります。

---------------------
Given the context information and not prior knowledge, answer the question: 履修時に気をつけることはありますか？



INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 173 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 27 tokens



> Finished chain.
 welcome to the answer to the faculty of education, is below.  i don’t get to replace, don’t get to performance, don’t get to a question to question.  get to a comfort: get influence to before primary: get influence to before as the question are infected with some degree limits.  questions of essay: 質問は
